<a href="https://colab.research.google.com/github/Srikanth421/NLP/blob/main/Fine_tuning_a_pretrained_model_finbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 24.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

In [3]:
raw_datasets = load_dataset("glue", "sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.6 MB/s eta 0:00:00


In [6]:
from transformers import AutoTokenizer

checkpoint = "ProsusAI/finbert"
# checkpoint = "cardiffnlp/twitter-roberta-base-sentiment"
# checkpoint = "sentence-transformers/stsb-roberta-base-v2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenized_sentence = tokenizer(raw_datasets["train"]["sentence"])
# tokenized_sentence = tokenizer(raw_datasets["train"]["sentence2"])

In [7]:
inputs = tokenizer("What is the best movie of 2022?", "There are many best movies!")
inputs

{'input_ids': [101, 2054, 2003, 1996, 2190, 3185, 1997, 16798, 2475, 1029, 102, 2045, 2024, 2116, 2190, 5691, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'what',
 'is',
 'the',
 'best',
 'movie',
 'of',
 '202',
 '##2',
 '?',
 '[SEP]',
 'there',
 'are',
 'many',
 'best',
 'movies',
 '!',
 '[SEP]']

In [9]:
tokenized_dataset = tokenizer(
    raw_datasets["train"]["sentence"],
    # raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True,
)

In [10]:
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

In [11]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
samples = tokenized_datasets["train"][:8]
samples = {k: v for k, v in samples.items() if k not in ["idx", "sentence"]}
[len(x) for x in samples["input_ids"]]

[10, 11, 15, 10, 22, 13, 29, 6]

In [14]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([8, 29]),
 'token_type_ids': torch.Size([8, 29]),
 'attention_mask': torch.Size([8, 29]),
 'labels': torch.Size([8])}

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments("test-trainer",max_steps= 12000)

In [17]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2,ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [19]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.408800
1000,0.382600
1500,0.337200
2000,0.318000
2500,0.321800
3000,0.295400
3500,0.315700
4000,0.280400
4500,0.257100
5000,0.270900


TrainOutput(global_step=12000, training_loss=0.24080152034759522, metrics={'train_runtime': 1092.3224, 'train_samples_per_second': 87.886, 'train_steps_per_second': 10.986, 'total_flos': 1466618022568800.0, 'train_loss': 0.24080152034759522, 'epoch': 1.43})

In [20]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(872, 2) (872,)


In [21]:
print(predictions[0])

[[-3.9979806  3.9875033]
 [ 3.387494  -3.1864505]
 [-3.7201533  3.6070106]
 ...
 [ 1.1927406 -2.0628693]
 [ 2.3485801 -2.872622 ]
 [ 1.2727648 -2.095231 ]]


In [22]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [23]:
print(preds[0:5],raw_datasets["validation"]["sentence"][0:5])

[1 0 1 1 0] ["it 's a charming and often affecting journey . ", 'unflinchingly bleak and desperate ', 'allows us to hope that nolan is poised to embark a major career as a commercial yet inventive filmmaker . ', "the acting , costumes , music , cinematography and sound are all astounding given the production 's austere locales . ", "it 's slow -- very , very slow . "]


In [24]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.5 MB/s eta 0:00:00


In [25]:
import evaluate

metric = evaluate.load("glue", "sst2")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.9162844036697247}

In [26]:
trainer.save_model("drive/MyDrive/finbert_12000_steps_model")

In [27]:
trainer.evaluate()

{'eval_loss': 0.388929545879364,
 'eval_runtime': 6.4318,
 'eval_samples_per_second': 135.576,
 'eval_steps_per_second': 16.947,
 'epoch': 1.43}